<a href="https://colab.research.google.com/github/mikhailklassen/CNN_MNIST/blob/main/MNIST_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

# In Google Colab, can change runtime to GPU if desired. If a TensorFlow 
# operation has both CPU and GPU implementations, by default the GPU device 
# is prioritized when the operation is assigned.
device = tf.config.get_visible_devices()
print(device)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# Random seed for reproducibility
seed = 42
tf.keras.utils.set_random_seed(seed)

# Save the model at the end?
save_model = False

# Batch sizes for training and testing
batch_size = 64
test_batch_size = 14

# Training epochs
n_epochs = 10

# Learning rate
learning_rate = 1.0

# Decay rate for adjusting the learning rate
gamma = 0.7

# Number of target classes in the MNIST data
num_classes = 10

# Data input shape
input_shape = (28, 28, 1)

In [4]:
# Load the MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# The scaled mean and standard deviation of the MNIST dataset (precalculated)
data_mean = 0.1307
data_std = 0.3081

# Reshape the input data
x_train = x_train.reshape(x_train.shape[0], 
                          x_train.shape[1], 
                          x_train.shape[2], 1)

x_test = x_test.reshape(x_test.shape[0], 
                        x_test.shape[1], 
                        x_test.shape[2], 1)

# Normalize the data
x_train = (x_train/255.0 - data_mean) / data_std
x_test = (x_test/255.0 - data_mean) / data_std

# Convert labels to one-hot vectors
y_train = tf.one_hot(y_train.astype(np.int32), depth=num_classes)
y_test = tf.one_hot(y_test.astype(np.int32), depth=num_classes)

11501568/11490434 [==============================] - 0s 0us/step


In [5]:
# Define the architecture of the neural network
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), strides=(1,1),
                                      padding='valid', 
                                      activation='relu',
                                      input_shape=input_shape),
    tf.keras.layers.Conv2D(64, (3,3), strides=(1,1),
                                      padding='valid',
                                      activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [6]:
# Decay the learning rate at a base rate of gamma roughly every epoch, which
# is len(x_train) steps
scheduler = tf.keras.optimizers.schedules.ExponentialDecay(
    learning_rate,
    decay_steps=len(x_train),
    decay_rate=gamma)

# Define the optimizer to user for gradient descent
optimizer = tf.keras.optimizers.Adadelta(scheduler)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])

# Display a model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 64)        0         
                                                                 
 flatten (Flatten)           (None, 9216)              0         
                                                                 
 dense (Dense)               (None, 128)               1179776   
                                                        

In [7]:
# Train the model
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=n_epochs,
          validation_data=(x_test, y_test),
          validation_batch_size=test_batch_size)

Epoch 1/10
938/938 [==============================] - 22s 14ms/step - loss: 0.1896 - acc: 0.9422 - val_loss: 0.0467 - val_acc: 0.9840
Epoch 2/10
938/938 [==============================] - 12s 13ms/step - loss: 0.0715 - acc: 0.9790 - val_loss: 0.0374 - val_acc: 0.9876
Epoch 3/10
938/938 [==============================] - 12s 13ms/step - loss: 0.0565 - acc: 0.9826 - val_loss: 0.0354 - val_acc: 0.9887
Epoch 4/10
938/938 [==============================] - 12s 13ms/step - loss: 0.0476 - acc: 0.9863 - val_loss: 0.0314 - val_acc: 0.9900
Epoch 5/10
938/938 [==============================] - 12s 13ms/step - loss: 0.0430 - acc: 0.9872 - val_loss: 0.0328 - val_acc: 0.9892
Epoch 6/10
938/938 [==============================] - 13s 13ms/step - loss: 0.0376 - acc: 0.9890 - val_loss: 0.0268 - val_acc: 0.9912
Epoch 7/10
938/938 [==============================] - 12s 12ms/step - loss: 0.0345 - acc: 0.9895 - val_loss: 0.0276 - val_acc: 0.9916
Epoch 8/10
938/938 [==============================] - 12s 13ms

In [8]:
if save_model:
    model.save_weights("mnist_cnn_tf.ckpt")